In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score,f1_score, confusion_matrix
import re
from nltk.stem import WordNetLemmatizer
from keras.preprocessing.text import Tokenizer
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/capstone/data/fake reviews dataset.csv")
data.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"Love this! Well made, sturdy, and very comfor..."
1,Home_and_Kitchen_5,5.0,CG,"love it, a great upgrade from the original. I..."
2,Home_and_Kitchen_5,5.0,CG,This pillow saved my back. I love the look and...
3,Home_and_Kitchen_5,1.0,CG,"Missing information on how to use it, but it i..."
4,Home_and_Kitchen_5,5.0,CG,Very nice set. Good quality. We have had the s...


In [ ]:
data['text_'][15]

'Absolutely adorable! And excellent price. We have had the wooden ones for a few months now and they'

# Data cleaning

In [ ]:
lemmatizer = WordNetLemmatizer()
stop_words= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't","not"])

In [ ]:
def preprocess(text):
  text=text.lower()
  text = [word for word in text.split(" ") if not word in stop_words] # removing stop words
  text= [lemmatizer.lemmatize(token, "v") for token in text] #Lemmatization
  text=" ".join(text) 
  return text
def decontract(text):
    text = re.sub(r"won\'t", "will not", text)
    text = re.sub(r"can\'t", "can not", text)
    text = re.sub(r"n\'t", " not", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'s", " is", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'t", " not", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'m", " am", text)
    return text

## Pre-processing

In [ ]:
data["text_"]=data["text_"].apply(lambda x:decontract(x))
data["text_"]=data["text_"].apply(lambda x:preprocess(x))
data.head()

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,CG,"love this! well made, sturdy, comfortable. l..."
1,Home_and_Kitchen_5,5.0,CG,"love it, great upgrade original. mine couple ..."
2,Home_and_Kitchen_5,5.0,CG,pillow save back. love look feel pillow.
3,Home_and_Kitchen_5,1.0,CG,"miss information use it, great product price!"
4,Home_and_Kitchen_5,5.0,CG,nice set. good quality. set two months


In [ ]:
from sklearn.preprocessing import LabelEncoder

In [ ]:
le=LabelEncoder()

In [ ]:
data["label"] = le.fit_transform(data["label"])


In [ ]:
data # 0 - computer generatede,  1 - original

,category,rating,label,text_
0,Home_and_Kitchen_5,5.0,0,"love this! well made, sturdy, comfortable. l..."
1,Home_and_Kitchen_5,5.0,0,"love it, great upgrade original. mine couple ..."
2,Home_and_Kitchen_5,5.0,0,pillow save back. love look feel pillow.
3,Home_and_Kitchen_5,1.0,0,"miss information use it, great product price!"
4,Home_and_Kitchen_5,5.0,0,nice set. good quality. set two months
...,...,...,...,...
40427,Clothing_Shoes_and_Jewelry_5,4.0,1,read review say bra run small order two band c...
40428,Clothing_Shoes_and_Jewelry_5,5.0,0,sure exactly would be. little large small size...
40429,Clothing_Shoes_and_Jewelry_5,2.0,1,"wear hood itself, wear hood, wear jacket witho..."
40430,Clothing_Shoes_and_Jewelry_5,1.0,0,like nothing dress. reason give 4 star order s...


In [ ]:
data['text_'][15]

'absolutely adorable! excellent price. wooden ones months'

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data['text_'], data['label'], test_size=0.3, random_state=1,shuffle =True)

In [ ]:
X_train

26635    know love author, hardship read book.  think w...
9327     exactly look for. quality good really best one...
10020    fantastic!! able drastically reduce amount spa...
39384    beautiful ring sure ip plated. durability high...
33139         fit nice lego brand.  would buy brand again.
                               ...                        
7813     difficult open close side bags.  really requir...
32511    henri charrier-papillon-a figure national psyc...
5192     zips-up high, well made, wide brim. keep next one
12172         decent movie, disappointed. movie slow. many
33003                    50/50 stay put. problem tiny, top
Name: text_, Length: 28302, dtype: object

## Preprocessing

In [ ]:
from keras_preprocessing.sequence import pad_sequences

In [ ]:
top_words = 6000    #top 6000 words in the vocabulary
max_review_length = 130  #maximum sentence length
embedding_vector_length = 32  #each word is mapped to a 32 dimensional vector
tokenizer = Tokenizer(num_words=top_words) # top 6000 are getting tokenized
tokenizer.fit_on_texts(X_train)
list_tokenized_train = tokenizer.texts_to_sequences(X_train) # text data is converted to sequence data

In [ ]:
X_train_pad = pad_sequences(list_tokenized_train, maxlen=max_review_length)  # making all input sequence into same length

#### One-Hot encoding

In [ ]:
from tensorflow.keras.preprocessing.text import one_hot

In [ ]:
vectorizer1 = TfidfVectorizer(ngram_range = (1,1))
vectorizer1.fit(X_train)
train_vectors=vectorizer1.transform(X_train)
train_vectors

<28302x32249 sparse matrix of type '<class 'numpy.float64'>'
	with 746235 stored elements in Compressed Sparse Row format>

In [ ]:
train_vectors.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
test_vectors=vectorizer1.transform(X_test)

## Bi-LSTM

In [ ]:
from keras.layers import Dense , Input , LSTM , Embedding, Dropout , Activation, GRU, Flatten
from keras.layers import Bidirectional, GlobalMaxPool1D
from keras.models import Model, Sequential
from keras.layers import Convolution1D
from keras import initializers, regularizers, constraints, optimizers, layers

In [ ]:
model = Sequential()  
model.add(Embedding(top_words+1, embedding_vector_length, input_length=max_review_length)) 
model.add(Bidirectional(LSTM(100)))
model.add(Dense(1, activation='relu'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 130, 32)           192032    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              106400    
 nal)                                                            
                                                                 
 dense_2 (Dense)             (None, 1)                 201       
                                                                 
Total params: 298,633
Trainable params: 298,633
Non-trainable params: 0
_________________________________________________________________


In [ ]:
history = model.fit(X_train_pad,y_train, epochs=30, batch_size=64)

Epoch 1/30
443/443 [==============================] - 11s 17ms/step - loss: 0.6054 - accuracy: 0.7422
Epoch 2/30
443/443 [==============================] - 8s 17ms/step - loss: 0.3711 - accuracy: 0.8643
Epoch 3/30
443/443 [==============================] - 7s 17ms/step - loss: 0.3579 - accuracy: 0.8682
Epoch 4/30
443/443 [==============================] - 7s 17ms/step - loss: 0.2957 - accuracy: 0.9014
Epoch 5/30
443/443 [==============================] - 8s 17ms/step - loss: 0.2670 - accuracy: 0.9170
Epoch 6/30
443/443 [==============================] - 8s 17ms/step - loss: 0.2694 - accuracy: 0.9188
Epoch 7/30
443/443 [==============================] - 8s 17ms/step - loss: 0.3153 - accuracy: 0.9019
Epoch 8/30
443/443 [==============================] - 8s 17ms/step - loss: 0.2318 - accuracy: 0.9289
Epoch 9/30
443/443 [==============================] - 9s 20ms/step - loss: 0.1904 - accuracy: 0.9429
Epoch 10/30
443/443 [==============================] - 8s 17ms/step - loss: 0.1746 - accur

In [ ]:
list_tokenized_test = tokenizer.texts_to_sequences(X_test)
X_test_pad = pad_sequences(list_tokenized_test, maxlen=max_review_length)
prediction = model.predict(X_test_pad)
y_pred = (prediction > 0.5)
print("Accuracy of the model : ", accuracy_score(y_pred, y_test))
print('F1-score: ', f1_score(y_pred, y_test))
print('Confusion matrix:')
confusion_matrix(y_test,y_pred)

380/380 [==============================] - 3s 6ms/step
Accuracy of the model :  0.8760923330585325
F1-score:  0.8801148600143575
Confusion matrix:


array([[5110,  997],
       [ 506, 5517]])

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
 print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.84      0.87      6107
           1       0.85      0.92      0.88      6023

    accuracy                           0.88     12130
   macro avg       0.88      0.88      0.88     12130
weighted avg       0.88      0.88      0.88     12130

